In [4]:
import pandas as pd 
import numpy as np 
from collections import Counter, defaultdict
import random
from diffprivlib.mechanisms import PermuteAndFlip
from Forest_7 import DPRF_Forest
import sys
import math

## Datasets - Adult, Mushroom, Heart (?)

In [5]:
# Missing Attribute Values: 2480 of them (denoted by "?"), all forattribute #11.

datasetmush = 'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
row_names = ["edible", "cap-shape", "cap-surf", "cap-col", "bruise", "odor", "gill-att", "gill-space", "gill-size",
"gill-col", "stalk-shape", "stalk-root", "st-surf-ab-ring", "st-surf-bl-ring", "st-col-ab-ring", "st-col-bl-ring",
"veil-type", "veil-col", "ring-num", "ring-type", "spore-col", "popul", "habitat"]
mushdata = pd.read_csv(datasetmush, names = row_names, na_values=[" ?"])
mushdata['edible'].replace(to_replace=['e', 'p'], value=[1, 0], inplace=True)
mushdata["edible"] = mushdata["edible"].astype(int)
Dmush = mushdata.to_numpy()

mushdata.shape

(8124, 23)

In [7]:
mushdata.head()

,edible,cap-shape,cap-surf,cap-col,bruise,odor,gill-att,gill-space,gill-size,gill-col,...,st-surf-bl-ring,st-col-ab-ring,st-col-bl-ring,veil-type,veil-col,ring-num,ring-type,spore-col,popul,habitat
0,0,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,1,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,1,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,0,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,1,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [8]:
mushdata["cap-shape"].value_counts().to_dict()
g =  {'x': 335, 'b': 41, 'f': 317, 'k': 7, 's': 4, 'c': 1, }

In [9]:
list(g.keys())

['x', 'b', 'f', 'k', 's', 'c']

In [10]:
train_data_mush.shape

NameError: name 'train_data_mush' is not defined

In [59]:
sum(g.values())

705

In [60]:
# g[el] -> |C|
# | X_i \ C |
# 
{el: g[el] - (sum(g.values()) - g[el]) for el in g.keys()}

{'x': -35, 'b': -623, 'f': -71, 'k': -691, 's': -697, 'c': -703}

In [11]:
mushdata = mushdata[["cap-shape", "cap-surf", "cap-col", "bruise", "odor", "gill-att", "gill-space", "gill-size",
"gill-col", "stalk-shape", "stalk-root", "st-surf-ab-ring", "st-surf-bl-ring", "st-col-ab-ring", "st-col-bl-ring",
"veil-type", "veil-col", "ring-num", "ring-type", "spore-col", "popul", "habitat", "edible"]]

In [12]:
mush = mushdata[mushdata["stalk-root"] != "?"]

In [13]:
np.random.seed(97)
D2mush = mush.to_numpy()
D2mush = D2mush.astype(str)
np.random.shuffle(D2mush)

In [24]:
dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [25]:
random.seed(42)
adult2 = us_adult_income.sample(frac=1, random_state=42)
adult2.shape

(32561, 15)

In [26]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [27]:
adult23 = adult25[["Age", #DONE
                    "Workclass", # DONE
                    "Education", #DONE
                    "MaritalStatus", # DONE
                    "Occupation", # DONE
                    "Relationship", # DONE
                    "Race", # DONE
                    "HoursPerWeek", # DONE
                    "Gender", # DONE
                    #"CapitalGain",
                    #"CapitalLoss",
                    "Income"]] # Education, Workclass, Occupation and Gender are discrete. 
#adult23

In [47]:
adult23['Workclass'].unique()

array([3, 6, 5, 4, 0, 1], dtype=int8)

In [28]:
adult23 = adult23.drop(adult23[(adult23["Workclass"] == -1) |
                               (adult23["Education"] == -1) |
                               (adult23["Occupation"] == -1) |
                               (adult23["MaritalStatus"] == -1) |
                                (adult23["HoursPerWeek"] == -1) 
                            ].index)
adult23.shape

(30718, 10)

In [ ]:
#adult23 = adult23[adult23["Country"] != 14]
#adult23["Country"].value_counts()

In [102]:
#adult23 = adult23[adult23["Occupation"] != 1]
#adult23 = adult23[adult23["Occupation"] != -1]
#adult23 = adult23[adult23["Occupation"] != 8]
#adult23["Occupation"].value_counts()

In [103]:
#adult23 = adult23[adult23["Education"] != 13]
#adult23 = adult23[adult23["Education"] != 3]
# adult23["Education"].value_counts()

In [29]:
adult23 = adult23[adult23["Workclass"] != 7]  
#adult23 = adult23[adult23["Workclass"] != 6]
# only 4 people in workclass 6...
adult23["Workclass"].value_counts()

3    22696
5     2541
1     2093
6     1298
4     1116
0      960
Name: Workclass, dtype: int64

In [16]:
#adult23 = adult23[adult23["Race"] != 0]  
#adult23 = adult23[adult23["Race"] != 3]
# only 4 people in workclass 6...
#adult23["Race"].value_counts()

In [17]:
#adult23 = adult23[adult23["MaritalStatus"] != 1]  
#adult23 = adult23[adult23["MaritalStatus"] != 3]
# only 4 people in workclass 6...
#adult23["MaritalStatus"].value_counts()

In [18]:
                    #"Workclass", # DONE
                    #"Education", #DONE
                    #"MaritalStatus", # DONE
                    #"Occupation", # DONE
                    #"Relationship", # DONE
                    #"Race", # DONE
                    #"Gender"
#adult23["Workclass"].value_counts()

In [30]:
np.random.seed(97)
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

In [31]:
D4 = D2.astype(str)
# D4

In [32]:
# make sure both test and training data have all classes in them
# the code will not run if D88 and D99 do not print the same arrays
D22 = D2
len(D22)

30704

In [33]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [34]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [31]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [35]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [36]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,1]).astype(int))

array([0, 1, 3, 4, 5, 6])

In [37]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,1]).astype(int))

array([0, 1, 3, 4, 5, 6])

In [38]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,3]).astype(int))

array([0, 1, 2, 3, 4, 5, 6])

In [39]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,3]).astype(int))

array([0, 1, 2, 3, 4, 5, 6])

https://github.com/loginaway/DecisionTree

In [41]:
# adult23[["CapitalGain", "CapitalLoss"]].describe()

In [18]:
len(D2)

30704

In [40]:
# from sklearn.model_selection import train_test_split

low = int(len(D2) * 0.75)
high = len(D2)

train_data = D2[0:low]
test_data = D2[low:high]

# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
print(len(test_data), len(train_data))
print(len(D2))
print(len(test_data) + len(train_data))

7676 23028
30704
30704


In [95]:
# D
# 'Workclass': '0',
train_data[:, 1]

array(['3', '6', '1', ..., '3', '3', '3'], dtype='<U21')

In [77]:
train_data[:, 0].astype(int)

array([37, 36, 53, ..., 27, 32, 23])

In [17]:
feature_discrete_mushroom = {"cap-shape":True, 
                            "cap-surf":True, 
                            "cap-col":True, 
                            "bruise":True, 
                            "odor":True, 
                            "gill-att":True, 
                            "gill-space":True, 
                            "gill-size":True, "gill-col":True, 
"stalk-shape":True, "stalk-root":True, "st-surf-ab-ring":True, "st-surf-bl-ring":True, "st-col-ab-ring":True, 
"st-col-bl-ring":True,"veil-type":True, "veil-col":True, "ring-num":True, "ring-type":True, "spore-col":True, 
"popul":True, "habitat":True}

low = int(len(D2mush) * 0.75)
high = len(D2mush)

train_data_mush = D2mush[0:low]
test_data_mush = D2mush[low:high]

# X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
print(len(test_data_mush), len(train_data_mush))
print(len(D2mush))
print(len(test_data_mush) + len(train_data_mush))


1411 4233
5644
5644


In [18]:
train_data_mush 

array([['x', 'f', 'g', ..., 'v', 'd', '1'],
       ['b', 'y', 'w', ..., 'n', 'g', '1'],
       ['f', 's', 'y', ..., 'v', 'd', '1'],
       ...,
       ['f', 'f', 'g', ..., 'y', 'd', '0'],
       ['x', 'f', 'g', ..., 'y', 'd', '1'],
       ['x', 'f', 'e', ..., 'y', 'd', '1']], dtype='<U1')

In [27]:
AA = {'cap-shape': [0], 'cap-surf': [1], 'cap-col': [2], 'bruise': [3], 'odor': [4], 'gill-att': [5], 'gill-space': [6], 'gill-size': [7], 'gill-col': [8], 'stalk-shape': [9], 'stalk-root': [10], 'st-surf-ab-ring': [11], 'st-surf-bl-ring': [12], 'st-col-ab-ring': [13], 'st-col-bl-ring': [14], 'veil-type': [15], 'veil-col': [16], 'ring-num': [17], 'ring-type': [18], 'spore-col': [19], 'popul': [20], 'habitat': [21]}
[el[1] for el in list(AA.items())]

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21]]

In [89]:
from Forest_22 import DPRF_Forest

In [90]:
# punainen
# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m
    # banana

B = 0.25 # epsilon
# f = 3
dm = 10
num_trees = 3


feature_discrete_adult = {'Age': False, 
                          'Workclass':True, 
                          'Education': True, 
                          "MaritalStatus":True,
                          'Occupation': True, 
                          "Relationship":True,
                          "Race":True,
                          "HoursPerWeek":False,
                          "Gender": True}
                          #"CapitalGain":False,
                          #"CapitalLoss":False}
                          
diff_priv_forest = DPRF_Forest(adult2, num_trees, adult23, train_data, B, dm, feature_discrete_adult)

####   diff_priv_forest2 = DPRF_Forest( mushdata, num_trees, mush, train_data_mush, B, dm, feature_discrete_mushroom )
                            #  A_ind,      A, attribute_values, root, class_values, feature_discrete, treetype, dataset_size, epsilon_per_tree, max_depth)

# RecursionError: maximum recursion depth exceeded while calling a Python object
# TRee_DPDR:
# node['>'] = self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A, self.current_depth+1)

#feature_discrete_iris =  {'sepal length': False, 'sepal width': False, 'petal length': False, 'petal width': False}
#diff_priv_forest = DPRF_Forest(irisdata, irisdata, Diris.astype('<U21'), B, f, dm, feature_discrete_iris)

#num_trees = len(our_diff_priv_forest._trees)
#av_prunings = np.mean([x._prunings for x in our_diff_priv_forest._trees])

#av_tree_size = np.mean([x._id-x._prunings+1 for x in our_diff_priv_forest._trees])
#accuracy, leafs_not_most_confident, votes_requiring_average = diff_priv_forest.evaluate_accuracy_with_voting(adult23, class_index=0) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

# feature discrete
# {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True}

NUM TREES = 3 & EPSILON PER TREE = 0.08333333333333333
Epsilon used since tree is random :  0.25
currently training! 
IN A
{'Age': [0], 'Workclass': [1], 'Education': [2], 'MaritalStatus': [3], 'Occupation': [4], 'Relationship': [5], 'Race': [6], 'HoursPerWeek': [7], 'Gender': [8]}
continuous attr:  Age
attr info:  [0]
age minmax:  17 90
Age split_points: 
[17.5 18.5 19.5 20.5 21.5 22.5 23.5 24.5 25.5 26.5 27.5 28.5 29.5 30.5
 31.5 32.5 33.5 34.5 35.5 36.5 37.5 38.5 39.5 40.5 41.5 42.5 43.5 44.5
 45.5 46.5 47.5 48.5 49.5 50.5 51.5 52.5 53.5 54.5 55.5 56.5 57.5 58.5
 59.5 60.5 61.5 62.5 63.5 64.5 65.5 66.5 67.5 68.5 69.5 70.5 71.5 72.5
 73.5 74.5 75.5 76.5 78.  79.5 80.5 81.5 82.5 83.5 86.  89. ]
cont scores:  {17.5: 3758, 18.5: 3642, 19.5: 3498, 20.5: 3338, 21.5: 3196, 22.5: 3024, 23.5: 2820, 24.5: 2620, 25.5: 2436, 26.5: 2256, 27.5: 2052, 28.5: 1862, 29.5: 1654, 30.5: 1446, 31.5: 1204, 32.5: 1030, 33.5: 818, 34.5: 570, 35.5: 348, 36.5: 108, 37.5: 84, 38.5: 278, 39.5: 484, 40.5: 668, 4

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'numpy.str_'

In [41]:
#g = D2[400:][:,-4]
#unique, counts = np.unique(g, return_counts=True)
#print(np.asarray((unique, counts)).T)

In [22]:
test_data_mush

array([['x', 's', 'g', ..., 's', 'g', '0'],
       ['x', 's', 'w', ..., 'n', 'g', '1'],
       ['f', 'f', 'y', ..., 'y', 'p', '0'],
       ...,
       ['x', 'y', 'e', ..., 'y', 'd', '1'],
       ['x', 'y', 'y', ..., 'n', 'm', '1'],
       ['x', 'y', 'e', ..., 'v', 'd', '1']], dtype='<U1')

In [23]:
#accuracy = diff_priv_forest.evaluate_accuracy_with_voting(test_data, class_index=11) # confidence count includes all trees (before voting)
accuracy = diff_priv_forest2.evaluate_accuracy_with_voting(test_data_mush, class_index=22)

print("accuracy = {}".format(accuracy))

result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['0']]
result:  [['0']]
votes: 
[0, 0, 0]
result:  [['1']]
result:  [['1']]
result:  [['0']]
votes: 
[1, 1, 0]
result:  [['1']]
result:  [['1']]
result:  [['1']]
votes: 
[1, 1, 1]
result:  [['0']]
result:  [['1']]


In [ ]:
# something's not right with the voting system! 
#accuracy = diff_priv_forest.evaluate_accuracy_with_voting(Diris.astype('<U21'), class_index=4) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

In [461]:
# %history -g -f hybridTree2019.ipynb

In [12]:
def median(nums):
    nums = sorted(nums)
    middle1 = (len(nums) - 1) // 2
    middle2 = len(nums) // 2
    return (nums[middle1] + nums[middle2]) / 2

print("1rst method: ", median(t["a"]))
print("2nd method: ", median2(t["a"]))

1rst method:  3.5
3
2nd method:  3.5


In [56]:
n = list(t.iloc[:,0])
n2 = np.sort(n)

print(n2)

def calcabs(l):
    return([ abs(l[i] - l[i+1]) for i in range(0, len(l) -1 )  ])


def calsumcabs(l):
    return([ abs(  sum(l[0:i]) - sum(l[i+1:len(l)])  ) for i in range(0, len(l) -1 )  ])

print(calsumcabs(n2))
print(calcabs(n2))

#calcabs(n2)

[1 3 3 4 6 7]
[23, 19, 13, 6, 4]
[2, 0, 1, 2, 1]


In [ ]:
https://stackoverflow.com/questions/251781/how-to-find-the-kth-largest-element-in-an-unsorted-array-of-length-n-in-on/251884#251884

def Select(A,n,i):
    # Divide input into ⌈n/5⌉ groups of size 5.

    # /* Partition on median-of-medians */
    medians = array of each group’s median.
    pivot = Select(medians, ⌈n/5⌉, ⌈n/10⌉)
    Left Array L and Right Array G = partition(A, pivot)

    /* Find ith element in L, pivot, or G */
    k = |L| + 1
    If i = k, return pivot
    If i < k, return Select(L, k-1, i)
    If i > k, return Select(G, n-k, i-k)

In [54]:
import pandas as pd
t = pd.DataFrame({"a": [1,6,4,3,7,3], "b":[5,7,3,4,6,1], "c":["h","h","t","g","t","t"], "y":[1,1,0,0,0,1]})
t

,a,b,c,y
0,1,5,h,1
1,6,7,h,1
2,4,3,t,0
3,3,4,g,0
4,7,6,t,0
5,3,1,t,1


In [55]:
d = {}
for el in list(t['c'].unique()):
    a = list(t[t['c'] == el]['y'])
    d[el] = a
    
print(d)

{'h': [1, 1], 't': [0, 0, 1], 'g': [0]}


In [58]:
import numpy as np
from scipy.stats import f_oneway 

f_oneway(d['h'], d['t'], d['g'])
# since p-value is more than 0.05, we don't accepth the alternate hyp. 
# Var c does not affect target y.  

F_onewayResult(statistic=1.8750000000000002, pvalue=0.2962962962962963)

In [68]:
import numpy as np 

def score_func(x, cat=False):

    l = np.sort(t[x])
    an = {}

    if cat:
        
        for el in np.unique(l):
            c = (l == el).sum()
            r = c / ( len(l) / c ) # wrong btw
            an[el] = r
            
    else:
        print("listassa: ", np.unique(l))
        for el in np.unique(l):

            a = [i for i in l if i < el]
            b = [i for i in l if i >= el]
            s = abs(len(a) - len(b))
            an[el] = s

    return an 

def exponential(options, sensitivity, epsilon, scores):

    #options = df[el].unique() 
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(-(epsilon/2) * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    print(probabilities)

    # Choose an element from options based on the probabilities
    return np.random.choice(options, 1, p=probabilities)[0]

In [82]:
def cand_splits(df):

    splt = {}

    for el in list(df.columns[:-1]):

        options = df[el].unique() 

        print(type(df[el][0]))
        if isinstance(df[el][0], str):
            sc = score_func(el, True)
            score = list(sc.values())
        else:
            print('here')
            sc = score_func(el)
            score = list(sc.values())
        
        print(options)
        ans = exponential(options, 1, 1, score)
        print("ans: ", ans)
        print("_________")
        
        splt[el] = ans

    return splt

rs = cand_splits(t)
print("rs: ", rs)

<class 'numpy.int64'>
here
listassa:  [1 3 4 6 7]
[1 6 4 3 7]
[0.08697608 0.14339932 0.38979977 0.23642551 0.14339932]
ans:  4
_________
<class 'numpy.int64'>
here
listassa:  [1 3 4 5 6 7]
[5 7 3 4 6 1]
[0.07034478 0.11597894 0.19121695 0.31526345 0.19121695 0.11597894]
ans:  1
_________
<class 'str'>
['h' 't' 'g']
[0.38475919 0.3395488  0.27569201]
ans:  h
_________
rs:  {'a': 4, 'b': 1, 'c': 'h'}


In [93]:
# pick attribute 
import collections

def attr_score_func(df, rs, cat=False):

    msan = {}

    for el in list(df.columns)[:-1]:

        print("colname: ", el)
        
        if el != 'c':
            val = list(df[el]) # values in the column
            sp = rs[el] # the split

            print(type(val), type(sp))

            mse = np.mean((val - sp)**2)
        else:
            val = list(df[el]) # values in the column
            sp = rs[el] # the split

            col = list(collections.Counter(val).values() )
            rc = np.int64(val.count(sp))

            print(type(col), type(rc))

            mse = np.mean((col - rc)**2)

        msan[el] = mse

    return msan


def attr_exponential(R, sensitivity, epsilon, scores):
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(-(epsilon/8) * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Choose an element from R based on the probabilities
    return np.random.choice(R, 1, p=probabilities)[0]


In [ ]:
(values - x)**2  -> score 
# MSE -> prediction on the split as the score 

In [96]:
def pick_split(df, rs):
        
    
    sc = attr_score_func(df, rs)

    # options = df[el].unique()
    score = list(sc.values())

    print('scores: ', score)
            
    ans = exponential(options, 1, 1, score)
    splt[el] = ans

    return score

rs = pick_split(t, rs)
print(rs)

colname:  a
<class 'list'> <class 'numpy.int64'>
colname:  b
<class 'list'> <class 'numpy.int64'>
colname:  c
<class 'list'> <class 'numpy.int64'>
scores:  [8.0, 5.666666666666667, 1.6666666666666667]
[8.0, 5.666666666666667, 1.6666666666666667]


In [28]:
# categorical
scc = score_func('c', True)
options = t['c'].unique() # continuous 
score = list(scc.values())

print("options:", options)
print('score: ', score)
# def exponential(R, sensitivity, epsilon, scores)
exponential(options, 1, 1, score)

options: ['h' 't' 'g']
score:  [0.16666666666666666, 0.6666666666666666, 1.5]
exp def probs:  [0.38475919 0.3395488  0.27569201]


't'

In [29]:
# continuous
scn = score_func('a')
options = t['a'].unique() # continuous 
score = list(scn.values())

print("options:", options)
print('score: ', score)
exponential(options, 1, 1, score)

options: [1 6 4 3 7]
score:  [6, 4, 0, 2, 4]
exp def probs:  [0.08697608 0.14339932 0.38979977 0.23642551 0.14339932]


4

In [81]:
t["c"]

un = t["c"].unique()
scor = [ abs(len(t[t["c"] == el]) - (t.shape[0] / len(t[t["c"] == el]))) for el in un]
scor

[1.0, 1.0, 5.0]

In [83]:
options = t["c"].unique()
exponential(options, 1, 1, scor)

[0.4223188 0.4223188 0.1553624]


'g'

In [10]:
from sklearn.metrics import mean_squared_error

u = mean_squared_error  # [0.375, 3.065, 1.875]

N = 3
B = 1 # [0, 1]

sens = 4*(B**2)/N 

print(sens)

def exponential(R, sensitivity, epsilon, scores): # x, u

    # Calculate the score for each element of R
    # scores = [u(x, r) for r in R]

    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(-(epsilon/2) * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    print(probabilities)

    # Choose an element from R based on the probabilities
    return np.random.choice(R, 1, p=probabilities)[0]

1.3333333333333333


In [12]:
# "a": [1,6,4,3,7,3], "b":[5,7,3,4,6,1], "c":["h","h","t","g","t","t"],
scor

[0.375, 3.065, 1.875]

In [17]:
import numpy as np 

opt =   ['a', 'b', 'c']

exponential( opt, sens, 1,  scor)

[0.42395831 0.25602117 0.32002052]


'a'

MSE formula = (1/n) * Σ(actual – forecast)2
Where:

    n = number of items (in the node),
    Σ = summation notation,
    Actual = original or observed y-value. (if this was a leaf node?)
    Forecast = y-value from regression (the class prediction)


Pr(a)  ∝ exp{ (-eN/8B^2) * (MSE(a)) }


MSE(a) : MSE associated with split 

In [2]:
from sklearn.metrics import mean_squared_error

y_true_a = [3, -0.5, 2, 7]
y_pred_a = [2.5, 0.0, 2, 8]

y_true_b = [3, -0.5, 2, 7]
y_pred_b = [1.5, -0.4, 3, 4]

y_true_c = [3, -0.5, 2, 7]
y_pred_c = [0.5, 0.0, 2, 6]


a = mean_squared_error(y_true_a, y_pred_a)
b = mean_squared_error(y_true_b, y_pred_b)
c = mean_squared_error(y_true_c, y_pred_c)

scor = [a, b, c]

In [3]:
scor

[0.375, 3.065, 1.875]